In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import numpy as np
import os
import random

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def count_training_samples(file_names,debug=False):
    num_train_samples = 0
    for file_name in file_names:
        x_train,y_train = read_numpy_file(file_name)
        assert len(x_train) == len(y_train), 'Number of samples must be same'
        if debug:
            print('X samples:{},Y samples:{}'.format(len(x_train),len(y_train)))
        num_train_samples = num_train_samples + len(x_train)
    return num_train_samples
        
def read_numpy_file(file_name):
    
    data = np.load(os.path.join(folder_name, file_name))
    #data = list(data)
    x_train = np.array([i[1] for i in data[:-10]])
    y_train = np.array([i[0] for i in data[:-10]])
    return x_train,y_train


In [45]:
folder_name_data = "./training_data_stage1"
folder_name_models = "./keras_models"
#folder_name = "./training_data_own"
file_names = os.listdir(folder_name_data)
random.shuffle(file_names)
file_names_training = file_names[0:int(len(file_names)*0.8)]
file_names_validation = file_names[-(len(file_names)-int(len(file_names)*0.8)):]
#num_train_samples = count_training_samples(file_names)
print('Number of files:',len(file_names))
print('Number of training files:',len(file_names_training))
print('Number of validation files:',len(file_names_validation))
#print('Total training examples:',num_train_samples)

Number of files: 2868
Number of training files: 2294
Number of validation files: 574


In [50]:
#Function to parse files for data (can be used for both training and validation data)
def _get_data_for_dataset(file_name):
    #print(file_name.decode())
    data = np.load(os.path.join(folder_name_data, file_name.decode()))
    #data = list(data)
    
    x_train = np.array([i[1] for i in data[:-10]]).reshape(-1, 176, 200, 3)
    y_train = np.array([i[0] for i in data[:-10]])
    return (x_train/255).astype('float32', copy=False),y_train.astype('float32', copy=False)

#Make dataset for training
dataset = tf.data.Dataset.from_tensor_slices(file_names_training)
dataset = dataset.flat_map(lambda file_name: tf.data.Dataset.from_tensor_slices(
    tuple (tf.py_func(_get_data_for_dataset, [file_name], [tf.float32,tf.float32]))))

dataset = dataset.shuffle(buffer_size=200)
dataset = dataset.batch(20) #Make dataset, shuffle, and create batches
#dataset= dataset.prefetch(1) 
dataset = dataset.repeat()

dataset_iterator = dataset.make_one_shot_iterator()
get_batch = dataset_iterator.get_next()

#Make dataset for validation
dataset_valid = tf.data.Dataset.from_tensor_slices(file_names_validation)
dataset_valid = dataset_valid.flat_map(lambda file_name: tf.data.Dataset.from_tensor_slices(
    tuple (tf.py_func(_get_data_for_dataset, [file_name], [tf.float32,tf.float32]))))

dataset_valid = dataset_valid.shuffle(buffer_size=200)
dataset_valid = dataset_valid.batch(20) #Make dataset, shuffle, and create batches
#dataset= dataset.prefetch(1) 
dataset_valid = dataset_valid.repeat()

dataset_valid_iterator = dataset.make_one_shot_iterator()
get_batch_valid = dataset_iterator.get_next()

In [6]:
#Hyper parameters
learning_rate = 1e-4

In [18]:
model_file_name = 'model.12-0.82.hdf5'
USE_EXISTING_MODEL = True
if os.path.isfile(os.path.join(folder_name_models, model_file_name)) and USE_EXISTING_MODEL == True:
    
    model =  tf.keras.models.load_model(os.path.join(folder_name_models, model_file_name))
    print('Loaded existing model named:{}'.format(model.name))
else:
    
    model = Sequential()
    #model.add(InputLayer(input_shape=(176,200,3),name='Input'))
    model.add(Conv2D(32,(3,3),padding='same',input_shape=(176,200,3),activation ='relu',name='Conv1'))
    model.add(Conv2D(32,(3,3),activation ='relu',name='Conv2'))              #Reduces size
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64,(3,3),padding='same',activation ='relu',name='Conv3'))
    model.add(Conv2D(64,(3,3),activation ='relu',name='Conv4'))           #Reduces size
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128,(3,3),padding='same',activation ='relu',name='Conv5'))
    model.add(Conv2D(128,(3,3),activation ='relu',name='Conv6'))           #Reduces size
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512,activation = 'relu',name='Dense1'))
    model.add(Dropout(0.2))
    model.add(Dense(4,activation = 'softmax',name='Output'))

    optimizer = tf.keras.optimizers.Adam(lr=learning_rate,decay=1e-6)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    model.compile(loss='categorical_crossentropy',optimizer = optimizer,metrics=['accuracy'])
    
    print('Created new model')

tensorboard = TensorBoard(log_dir='logs/stage1')
checkpointer = ModelCheckpoint(filepath='./keras_models/model.{epoch:02d}-{loss:.3f}-{acc:.2f}-{val_acc:.2f}.hdf5',monitor='acc',period=1)
model.summary()

Loaded existing model named:sequential
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 176, 200, 32)      896       
_________________________________________________________________
Conv2 (Conv2D)               (None, 174, 198, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 87, 99, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 87, 99, 32)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 87, 99, 64)        18496     
_________________________________________________________________
Conv4 (Conv2D)               (None, 85, 97, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4

In [56]:
epochs = 5
model.fit(dataset, epochs=epochs, steps_per_epoch=5000,validation_data=dataset_valid,validation_steps =1000,verbose=1, callbacks=[tensorboard,checkpointer])
model.save("BasicCNN-{}-epochs-{}-LR-STAGE1".format(epochs, learning_rate))
print('Model saved')

Epoch 1/5
5000/5000 [==============================] - 2041s 408ms/step - loss: 0.8989 - acc: 0.6257 - val_loss: 1.0879 - val_acc: 0.5512
Epoch 2/5
5000/5000 [==============================] - 2002s 400ms/step - loss: 0.8092 - acc: 0.6687 - val_loss: 1.1108 - val_acc: 0.5313
Epoch 3/5
5000/5000 [==============================] - 1998s 400ms/step - loss: 0.7021 - acc: 0.7175 - val_loss: 1.1647 - val_acc: 0.5125
Epoch 4/5
5000/5000 [==============================] - 1997s 399ms/step - loss: 0.6153 - acc: 0.7580 - val_loss: 1.3194 - val_acc: 0.4955
Epoch 5/5
5000/5000 [==============================] - 1994s 399ms/step - loss: 0.5438 - acc: 0.7902 - val_loss: 1.3615 - val_acc: 0.4810
Model saved
